# 네이버 검색광고 데이터 수집

이 노트북은 네이버 검색광고 API를 사용하여 광고 데이터를 수집합니다.

## 필요한 라이브러리 설치
```bash
pip install nevada
pip install pandas
```

In [31]:
# 필요한 라이브러리 import
import pandas as pd
import os
from datetime import datetime, timedelta

# nevada 라이브러리 import
try:
    from nevada.Common.Connector import Connector
    from nevada.API.Campaign import Campaign
    from nevada.API.StatReport import StatReport
    print("Nevada 라이브러리 import 성공")
except ImportError as e:
    print(f"Nevada 라이브러리 import 실패: {e}")
    print("Nevada 라이브러리가 설치되지 않았습니다. 'pip install nevada' 명령어로 설치해주세요.")

Nevada 라이브러리 import 성공


## API 정보 로드

API 정보를 data/info.csv 파일에서 로드합니다.

In [32]:
# API 정보 파일 읽기
api_info_path = "../data/info.csv"

try:
    # CSV 파일 읽기
    api_info = pd.read_csv(api_info_path)
    print("API 정보 파일 로드 성공")
    print(api_info.head())
    
    # 네이버 검색광고 정보 추출
    nsa_info = api_info[api_info['media'] == 'nsa'].iloc[0]
    
    api_key = nsa_info['key']
    secret_key = nsa_info['scr']
    customer_id = nsa_info['customer_id']
    
    print(f"Customer ID: {customer_id}")
    print("API Key와 Secret Key가 성공적으로 로드되었습니다.")
    
except FileNotFoundError:
    print(f"API 정보 파일을 찾을 수 없습니다: {api_info_path}")
except Exception as e:
    print(f"API 정보 로드 중 오류 발생: {e}")

API 정보 파일 로드 성공
  media                                                key  \
0   nsa  0100000000d2b4c68997cbb61c644799a15e20e51b275e...   

                                                 scr  customer_id  
0  AQAAAADStMaJl8u2HGRHmaFeIOUbG98LZQwLooUulIcFEN...      2398132  
Customer ID: 2398132
API Key와 Secret Key가 성공적으로 로드되었습니다.


## 네이버 검색광고 API 연결

In [33]:
# 네이버 검색광고 API 연결
base_url = 'https://api.naver.com'

try:
    # Connector 객체 생성
    conn = Connector(base_url, api_key, secret_key, customer_id)
    
    # 연결 테스트
    current_time = conn.get_datetime()
    print(f"API 연결 성공! 현재 시간: {current_time}")
    
except Exception as e:
    print(f"API 연결 실패: {e}")
    conn = None

API 연결 성공! 현재 시간: 2025-11-02 20:52:28.193702


## 캠페인 데이터 수집

In [34]:
# 캠페인 목록 조회
if conn:
    try:
        # Campaign 객체 생성
        campaign_api = Campaign(base_url, api_key, secret_key, customer_id)

        # 먼저 특정 캠페인 ID가 있는지 확인하고, 없다면 빈 데이터프레임 생성
        print("캠페인 데이터 수집을 위해서는 캠페인 ID가 필요합니다.")
        print("실제 캠페인 ID를 확인한 후 아래 코드를 수정하세요.")

        # 예시: 특정 캠페인 ID들 (실제 캠페인 ID로 변경 필요)
        sample_campaign_ids = ['cmp-a001-01-000000007564333','cmp-a001-01-000000008458832','cmp-a001-01-000000009357393']
        
        if sample_campaign_ids:
            # 여러 캠페인 정보 가져오기
            campaigns = campaign_api.list_by_ids(sample_campaign_ids)
            
            if campaigns:
                print(f"총 {len(campaigns)} 개의 캠페인을 찾았습니다.")
                
                # 캠페인 데이터를 DataFrame으로 변환
                campaigns_df = pd.DataFrame(campaigns)
                print("\n캠페인 정보:")
                print(campaigns_df.head())
            else:
                print("캠페인 데이터를 찾을 수 없습니다.")
                campaigns_df = pd.DataFrame()
        else:
            print("캠페인 ID 목록이 비어있습니다. 실제 캠페인 ID를 추가해주세요.")
            campaigns_df = pd.DataFrame()
            
    except Exception as e:
        print(f"캠페인 데이터 수집 중 오류 발생: {e}")
        campaigns_df = pd.DataFrame()
else:
    print("API 연결이 되지 않아 캠페인 데이터를 수집할 수 없습니다.")
    campaigns_df = pd.DataFrame()

캠페인 데이터 수집을 위해서는 캠페인 ID가 필요합니다.
실제 캠페인 ID를 확인한 후 아래 코드를 수정하세요.
총 3 개의 캠페인을 찾았습니다.

캠페인 정보:
                 nccCampaignId  customerId           name  userLock  \
0  cmp-a001-01-000000007564333     2398132  #1파워링크  턱관절MO     False   
1  cmp-a001-01-000000008458832     2398132           일반진료     False   
2  cmp-a001-01-000000009357393     2398132     파워링크250507     False   

  campaignTp deliveryMethod        trackingMode  usePeriod  dailyBudget  \
0   WEB_SITE    ACCELERATED  AUTO_TRACKING_MODE      False       400000   
1   WEB_SITE    ACCELERATED  AUTO_TRACKING_MODE      False       300000   
2   WEB_SITE    ACCELERATED  AUTO_TRACKING_MODE      False       350000   

   useDailyBudget  totalChargeCost    status statusReason  expectCost  \
0            True           134475  ELIGIBLE     ELIGIBLE        5170   
1            True                0  ELIGIBLE     ELIGIBLE          88   
2            True                0  ELIGIBLE     ELIGIBLE        2673   

   migType  delFlag           

## 성과 데이터 수집

In [35]:
# 성과 데이터 수집 (StatReport 사용)
if conn:
    try:
        # StatReport 객체 생성
        stat_report = StatReport(base_url, api_key, secret_key, customer_id)
        
        # 날짜 설정 (어제)
        report_date = (datetime.now() - timedelta(days=1)).strftime('%Y%m%d')
        
        print(f"보고서 생성 날짜: {report_date}")
        
        # 기존 보고서 목록 확인
        try:
            existing_reports = stat_report.list()
            print(f"기존 보고서 수: {len(existing_reports) if existing_reports else 0}")
        except Exception as e:
            print(f"기존 보고서 조회 실패: {e}")
            existing_reports = []
        
        # 새 보고서 생성 - AD 타입으로 시도
        print("새 보고서 생성 중... (AD 타입)")
        try:
            report_job = stat_report.create(reportTp='AD', statDt=report_date)
            
            if report_job and 'reportJobId' in report_job:
                report_job_id = report_job['reportJobId']
                print(f"보고서 작업 ID: {report_job_id}")
                print(f"보고서 상태: {report_job.get('status', 'Unknown')}")
                
                # reportJobId 타입 확인 및 변환
                if isinstance(report_job_id, int):
                    report_job_id = str(report_job_id)
                
                # 보고서 상태 확인
                try:
                    report_details = stat_report.get(reportJobId=report_job_id)
                    print(f"\n보고서 세부 정보:")
                    print(report_details)
                    
                    # 보고서 정보를 DataFrame으로 저장
                    performance_df = pd.DataFrame([{
                        'report_job_id': report_job_id,
                        'status': report_job.get('status', 'Unknown'),
                        'report_type': 'AD',
                        'report_date': report_date,
                        'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                        'details': str(report_details) if report_details else None
                    }])
                    
                except Exception as e:
                    print(f"보고서 상세 조회 실패: {e}")
                    # 기본 정보만으로 DataFrame 생성
                    performance_df = pd.DataFrame([{
                        'report_job_id': report_job_id,
                        'status': report_job.get('status', 'Unknown'),
                        'report_type': 'AD',
                        'report_date': report_date,
                        'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                        'error': str(e)
                    }])
                    
            else:
                print("보고서 생성에 실패했습니다.")
                print(f"응답: {report_job}")
                performance_df = pd.DataFrame()
                
        except Exception as e:
            print(f"보고서 생성 실패: {e}")
            print("다른 방법으로 데이터 수집을 시도합니다...")
            
            # 기본 연결 테스트
            try:
                current_time = conn.get_datetime()
                print(f"API 연결 상태: 정상 (현재 시간: {current_time})")
                
                # 간단한 더미 데이터로 DataFrame 생성
                performance_df = pd.DataFrame({
                    'date': [report_date],
                    'status': ['connection_ok'],
                    'note': ['StatReport API 호출 실패, 연결은 정상'],
                    'api_time': [str(current_time)]
                })
                
            except Exception as conn_e:
                print(f"연결 테스트도 실패: {conn_e}")
                performance_df = pd.DataFrame()
            
    except Exception as e:
        print(f"성과 데이터 수집 중 전체 오류 발생: {e}")
        performance_df = pd.DataFrame()
else:
    print("API 연결이 되지 않아 성과 데이터를 수집할 수 없습니다.")
    performance_df = pd.DataFrame()

# 수집된 데이터 확인
if not performance_df.empty:
    print(f"\n성과 데이터 수집 성공: {len(performance_df)} 건")
    print("\n수집된 데이터:")
    print(performance_df)

보고서 생성 날짜: 20251101
기존 보고서 수: 1
새 보고서 생성 중... (AD 타입)
보고서 작업 ID: 5279073
보고서 상태: REGIST

보고서 세부 정보:
{'reportJobId': 5279073, 'statDt': '2025-10-31T15:00:00Z', 'updateTm': '2025-10-31T15:00:00Z', 'reportTp': 'AD', 'status': 'RUNNING', 'downloadUrl': '', 'regTm': '2025-11-02T11:52:28Z', 'loginId': 'yoonnam220:naver'}

성과 데이터 수집 성공: 1 건

수집된 데이터:
  report_job_id  status report_type report_date           created_at  \
0       5279073  REGIST          AD    20251101  2025-11-02 20:52:29   

                                             details  
0  {'reportJobId': 5279073, 'statDt': '2025-10-31...  


## 데이터 저장

In [36]:
# 수집된 데이터를 CSV 파일로 저장
output_dir = "../data/naver_search_ad"
os.makedirs(output_dir, exist_ok=True)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# 캠페인 데이터 저장
if not campaigns_df.empty:
    campaigns_file = f"{output_dir}/campaigns_{timestamp}.csv"
    campaigns_df.to_csv(campaigns_file, index=False, encoding='utf-8-sig')
    print(f"캠페인 데이터 저장 완료: {campaigns_file}")

# 성과 데이터 저장
if not performance_df.empty:
    performance_file = f"{output_dir}/performance_{timestamp}.csv"
    performance_df.to_csv(performance_file, index=False, encoding='utf-8-sig')
    print(f"성과 데이터 저장 완료: {performance_file}")

print("\n데이터 수집 및 저장이 완료되었습니다.")

캠페인 데이터 저장 완료: ../data/naver_search_ad/campaigns_20251102_205229.csv
성과 데이터 저장 완료: ../data/naver_search_ad/performance_20251102_205229.csv

데이터 수집 및 저장이 완료되었습니다.


## 데이터 요약 및 분석

In [37]:
# 수집된 데이터 요약
print("=== 데이터 수집 요약 ===")
print(f"캠페인 수: {len(campaigns_df) if not campaigns_df.empty else 0}")
print(f"성과 데이터 수: {len(performance_df) if not performance_df.empty else 0}")

# 성과 데이터 기본 통계
if not performance_df.empty:
    print("\n=== 성과 데이터 기본 통계 ===")
    
    # 주요 지표별 통계 (컬럼이 존재하는 경우에만)
    numeric_columns = performance_df.select_dtypes(include=['number']).columns
    
    if len(numeric_columns) > 0:
        print(performance_df[numeric_columns].describe())
    else:
        print("수치형 데이터가 없습니다.")

# 캠페인 데이터 기본 정보
if not campaigns_df.empty:
    print("\n=== 캠페인 데이터 정보 ===")
    print(f"컬럼: {list(campaigns_df.columns)}")
    print(f"데이터 타입:\n{campaigns_df.dtypes}")

=== 데이터 수집 요약 ===
캠페인 수: 3
성과 데이터 수: 1

=== 성과 데이터 기본 통계 ===
수치형 데이터가 없습니다.

=== 캠페인 데이터 정보 ===
컬럼: ['nccCampaignId', 'customerId', 'name', 'userLock', 'campaignTp', 'deliveryMethod', 'trackingMode', 'usePeriod', 'dailyBudget', 'useDailyBudget', 'totalChargeCost', 'status', 'statusReason', 'expectCost', 'migType', 'delFlag', 'regTm', 'editTm', 'sharedBudgetLock']
데이터 타입:
nccCampaignId       object
customerId           int64
name                object
userLock              bool
campaignTp          object
deliveryMethod      object
trackingMode        object
usePeriod             bool
dailyBudget          int64
useDailyBudget        bool
totalChargeCost      int64
status              object
statusReason        object
expectCost           int64
migType              int64
delFlag               bool
regTm               object
editTm              object
sharedBudgetLock      bool
dtype: object


## 사용법 안내

### Nevada 라이브러리 사용 시 주의사항:

1. **캠페인 데이터 수집**: 캠페인 ID가 필요합니다. 
   - 네이버 검색광고 시스템에서 캠페인 ID를 확인한 후 `sample_campaign_ids` 리스트에 추가하세요.
   - 예시: `sample_campaign_ids = ['cmp-a001-01-000000002696103', 'cmp-a001-03-000000002696246']`

2. **통계 보고서**: StatReport를 사용하여 대용량 보고서를 생성할 수 있습니다.
   - 보고서 생성은 비동기로 처리되므로 상태를 확인해야 합니다.
   - 보고서 타입: 'CAMPAIGN', 'ADGROUP', 'AD' 등

3. **API 제한사항**: 네이버 검색광고 API는 일일 호출 제한이 있을 수 있습니다.

### 실제 사용 시 수정할 부분:
- `sample_campaign_ids` 리스트에 실제 캠페인 ID 추가
- 필요에 따라 보고서 타입 변경 (`reportTp` 파라미터)
- 날짜 범위 조정